In [1]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os
bearer_token = os.environ.get("bearer-token")

In [3]:
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAAHalVAEAAAAAVViwLGyKNUuT6ssD8tDTqiyQ1Pw%3DVXK9D7qXXABAcb96CPXxbltVZy3C9mzONePX8nLr9mkN3oHa7u'

In [4]:
def auth():
    return os.environ.get("TOKEN")

In [5]:
def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

In [6]:
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [7]:
def create_url(keyword, start_date, end_date, next_token = None):
    
    search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword, 
                    'start_time' : start_time, 
                    'end_time' : end_time,
                    'max_results' : max_results,
                    'tweet.fields': 'created_at',
                    'next_token': {}
                     }
    
    return (search_url, query_params)

In [8]:
def connect_to_endpoint(url, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.get( url, auth=bearer_oauth, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
#Inputs for tweets
#bearer_token = auth()
#headers = create_headers(bearer_token)
keyword = "amazon lang:en"
start_list =    ['2021-10-01T00:00:00.000Z', 
                    '2021-10-02T00:00:00.000Z', 
                    '2021-10-03T00:00:00.000Z',
                    '2021-10-04T00:00:00.000Z',
                    '2021-10-05T00:00:00.000Z',
                    '2021-10-06T00:00:00.000Z',
                    '2021-10-07T00:00:00.000Z',
                    '2021-10-08T00:00:00.000Z',
                    '2021-10-09T00:00:00.000Z',
                    '2021-10-10T00:00:00.000Z',
                    '2021-10-11T00:00:00.000Z',
                    '2021-10-12T00:00:00.000Z',
                    '2021-10-13T00:00:00.000Z',
                    '2021-10-14T00:00:00.000Z',
                    '2021-10-15T00:00:00.000Z',
                    '2021-10-16T00:00:00.000Z',
                    '2021-10-17T00:00:00.000Z',
                    '2021-10-18T00:00:00.000Z',
                    '2021-10-19T00:00:00.000Z',
                    '2021-10-20T00:00:00.000Z',
                    '2021-10-21T00:00:00.000Z',
                    '2021-10-22T00:00:00.000Z',
                    '2021-10-23T00:00:00.000Z',
                    '2021-10-24T00:00:00.000Z',
                    '2021-10-25T00:00:00.000Z',
                    '2021-10-26T00:00:00.000Z',
                    '2021-10-27T00:00:00.000Z',
                    '2021-10-28T00:00:00.000Z',
                    '2021-10-29T00:00:00.000Z',
                    '2021-10-30T00:00:00.000Z',
                    '2021-10-31T00:00:00.000Z'
                ]

end_list =    ['2021-10-01T23:59:59.000Z',
                    '2021-10-02T23:59:59.000Z',
                    '2021-10-03T23:59:59.000Z',
                    '2021-10-04T23:59:59.000Z',
                    '2021-10-05T23:59:59.000Z',
                    '2021-10-06T23:59:59.000Z',
                    '2021-10-07T23:59:59.000Z',
                    '2021-10-08T23:59:59.000Z',
                    '2021-10-09T23:59:59.000Z',
                    '2021-10-10T23:59:59.000Z',
                    '2021-10-11T23:59:59.000Z',
                    '2021-10-12T23:59:59.000Z',
                    '2021-10-13T23:59:59.000Z',
                    '2021-10-14T23:59:59.000Z',
                    '2021-10-15T23:59:59.000Z',
                    '2021-10-16T23:59:59.000Z',
                    '2021-10-17T23:59:59.000Z',
                    '2021-10-18T23:59:59.000Z',
                    '2021-10-19T23:59:59.000Z',
                    '2021-10-20T23:59:59.000Z',
                    '2021-10-21T23:59:59.000Z',
                    '2021-10-22T23:59:59.000Z',
                    '2021-10-23T23:59:59.000Z',
                    '2021-10-24T23:59:59.000Z',
                    '2021-10-25T23:59:59.000Z',
                    '2021-10-26T23:59:59.000Z',
                    '2021-10-27T23:59:59.000Z',
                    '2021-10-28T23:59:59.000Z',
                    '2021-10-29T23:59:59.000Z',
                    '2021-10-30T23:59:59.000Z',
                    '2021-10-31T23:59:59.000Z'
                ]

max_results = 540

#Total number of tweets we collected from the loop
total_tweets = 0

# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

#Create headers for the data you want to save, in this example, we only want save these columns in our dataset
csvWriter.writerow(['created_at','id','tweet'])
csvFile.close()

for i in range(0,len(start_list)):

    # Inputs
    count = 0 # Counting tweets per time period
    max_count = 100 # Max tweets per time period
    flag = True
    next_token = None
    
    # Check if flag is true
    while flag:
        # Check if max_count reached
        if count >= max_count:
            break
        print("-------------------")
        print("Token: ", next_token)
        url = create_url(keyword, start_list[i],end_list[i])
        json_response = connect_to_endpoint(url[0], url[1], next_token)
        result_count = json_response['meta']['result_count']

        if 'next_token' in json_response['meta']:
            # Save the token to use for next call
            next_token = json_response['meta']['next_token']
            print("Next Token: ", next_token)
            if result_count is not None and result_count > 0 and next_token is not None:
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)                
        # If no next token exists
        else:
            if result_count is not None and result_count > 0:
                print("-------------------")
                print("Start Date: ", start_list[i])
                append_to_csv(json_response, "data.csv")
                count += result_count
                total_tweets += result_count
                print("Total # of Tweets added: ", total_tweets)
                print("-------------------")
                time.sleep(5)
            
            #Since this is the final request, turn flag to false to move to the next time period.
            flag = False
            next_token = None
        time.sleep(5)
print("Total number of results: ", total_tweets)